In [1]:
from dotenv import load_dotenv
import os 

load_dotenv()

True

## <font color=yellow> Chat Completions API vs Assistants API</font>

**Chat Completions API**의 기본 단위는 `Messages`이며, 여기에 `Model`(`gpt-3.5-turbo`, `gpt-4` 등)을 사용하여 `Completion`을 수행함 
- 이 API는 가볍고 강력하지만 본질적으로 상태가 없기 때문에 대화 상태, 도구 정의, 검색 문서, 코드 실행을 수동으로 관리해야 함

**Assistants API**의 기본 단위는 다음과 같음
- 기본 모델, 지침, 도구, 문서(문맥)를 포함하는 `Assistants`,
- 대화의 상태를 나타내는 `Threads`,
- 텍스트 응답 및 다단계 도구 사용을 포함하여 `Thread`에서 `Assistant`의 실행을 구동하는 `Runs`.
    - 특정 Thread 안에서 Assistant가 실제로 동작하도록 요청함
    - 답변 생성, 도구 사용 등을 포함

전체 흐름
- Assistant 생성 → 어떤 비서인지 설정
- Thread 시작 → 대화 시작
- Run 요청 → 실제로 Assistant가 동작해 응답함

In [2]:
import json

def show_json(obj):
    display(json.loads(obj.model_dump_json()))

## <font color=yellow>1. Assistants 만들기</font>
- Assistants를 홈페이지에서 만들 수도 있고, 아래와 같이도 만들 수 있음

In [3]:
from openai import OpenAI

client = OpenAI()

assistant = client.beta.assistants.create(
    name="Math Tutor",
    instructions="너는 개인 수학 교사야. 질문에 한 문장 이하로 짧게 답해줘",
    model="gpt-4o",
)
show_json(assistant)

{'id': 'asst_QmH9Sj6swOlyTHjkn7VMhekO',
 'created_at': 1753255983,
 'description': None,
 'instructions': '너는 개인 수학 교사야. 질문에 한 문장 이하로 짧게 답해줘',
 'metadata': {},
 'model': 'gpt-4o',
 'name': 'Math Tutor',
 'object': 'assistant',
 'tools': [],
 'response_format': 'auto',
 'temperature': 1.0,
 'tool_resources': {'code_interpreter': None, 'file_search': None},
 'top_p': 1.0,
 'reasoning_effort': None}

In [5]:
assistant

Assistant(id='asst_QmH9Sj6swOlyTHjkn7VMhekO', created_at=1753255983, description=None, instructions='너는 개인 수학 교사야. 질문에 한 문장 이하로 짧게 답해줘', metadata={}, model='gpt-4o', name='Math Tutor', object='assistant', tools=[], response_format='auto', temperature=1.0, tool_resources=ToolResources(code_interpreter=None, file_search=None), top_p=1.0, reasoning_effort=None)

In [4]:
assistant.model_dump()

{'id': 'asst_QmH9Sj6swOlyTHjkn7VMhekO',
 'created_at': 1753255983,
 'description': None,
 'instructions': '너는 개인 수학 교사야. 질문에 한 문장 이하로 짧게 답해줘',
 'metadata': {},
 'model': 'gpt-4o',
 'name': 'Math Tutor',
 'object': 'assistant',
 'tools': [],
 'response_format': 'auto',
 'temperature': 1.0,
 'tool_resources': {'code_interpreter': None, 'file_search': None},
 'top_p': 1.0,
 'reasoning_effort': None}

## <font color=yellow>2. Threads 만들기</font>
- Assistant는 여러개의 Threads를 만들 수 있음
- 한 사람과 대화하는게 아니기 때문에

In [4]:
thread = client.beta.threads.create()
show_json(thread)

C:\Users\jjsd4\AppData\Local\Temp\ipykernel_32260\480294517.py:1: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  thread = client.beta.threads.create()


{'id': 'thread_D97HMuorDUdCSxehSzzucm02',
 'created_at': 1753185038,
 'metadata': {},
 'object': 'thread',
 'tool_resources': {'code_interpreter': None, 'file_search': None}}

In [5]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="방정식 '3x + 11 = 14'를 풀어줘"
)
show_json(message)

C:\Users\jjsd4\AppData\Local\Temp\ipykernel_32260\2549837072.py:1: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  message = client.beta.threads.messages.create(


{'id': 'msg_KMG3hwEwYfIbQJTyp4MFuY7j',
 'assistant_id': None,
 'attachments': [],
 'completed_at': None,
 'content': [{'text': {'annotations': [], 'value': "방정식 '3x + 11 = 14'를 풀어줘"},
   'type': 'text'}],
 'created_at': 1753185047,
 'incomplete_at': None,
 'incomplete_details': None,
 'metadata': {},
 'object': 'thread.message',
 'role': 'user',
 'run_id': None,
 'status': None,
 'thread_id': 'thread_D97HMuorDUdCSxehSzzucm02'}

## <font color=yellow>3. Runs</font>
- Threads를 Assistant와 연결
- Thread에 대한 Assistant의 Completion을 얻으려면 Run을 생성해야함
- Run을 생성하면 Assistant에게 Thread의 메시지를 살펴보고 조취를 취하라는 지시가 됨
    - 단일 응답을 추가하거나 도구를 사용할 수 있음

Runs는 Assistants API와 Chat Completions API 사이의 주요 차이점
- Chat Completions에서는 모델이 단일 메시지로만 응답할 수 있는 반면, Assistants API에서는 Run을 통해 Assistant가 하나 또는 여러 도구를 사용하고 Thread에 여러 메시지를 추가 할 수 있음

In [6]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
)
show_json(run)

C:\Users\jjsd4\AppData\Local\Temp\ipykernel_32260\3837867860.py:1: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run = client.beta.threads.runs.create(


{'id': 'run_VM0izCrE2IAuBcYNdEtUSkWG',
 'assistant_id': 'asst_ABxisLXuhXXBuVLYrk3sJJlR',
 'cancelled_at': None,
 'completed_at': None,
 'created_at': 1753185064,
 'expires_at': 1753185664,
 'failed_at': None,
 'incomplete_details': None,
 'instructions': '너는 개인 수학 교사야. 질문에 한 문장 이하로 짧게 답해줘',
 'last_error': None,
 'max_completion_tokens': None,
 'max_prompt_tokens': None,
 'metadata': {},
 'model': 'gpt-4o',
 'object': 'thread.run',
 'parallel_tool_calls': True,
 'required_action': None,
 'response_format': 'auto',
 'started_at': None,
 'status': 'queued',
 'thread_id': 'thread_D97HMuorDUdCSxehSzzucm02',
 'tool_choice': 'auto',
 'tools': [],
 'truncation_strategy': {'type': 'auto', 'last_messages': None},
 'usage': None,
 'temperature': 1.0,
 'top_p': 1.0,
 'tool_resources': {},
 'reasoning_effort': None}

Chat Completions API에서 완성을 생성하는 것과 달리, Run을 생성하는 것은 비동기 작업임 
- 이 작업은 Run의 메타데이터와 함께 즉시 반환되며, 여기에는 처음에 `queued`로 설정된 `status`가 포함됨 
- Assistant가 도구 사용과 메시지 추가와 같은 작업을 수행함에 따라 `status`가 업데이트됨

Assistant가 처리를 완료했는지 알기 위해, 우리는 Run을 반복적으로 폴링할 수 있습니다. (스트리밍 지원이 곧 제공될 예정)
- 여기서는 `queued` 또는 `in_progress` 상태만 확인하지만, 실제로 Run은 사용자에게 표시할 수 있는 [다양한 상태 변경](https://platform.openai.com/docs/api-reference/runs/object#runs/object-status)을 겪을 수 있음 
    - (이것들을 Steps라고 하며 나중에 다룰 것입니다.)
    - 완료가 되면 queued, in_progress가 아닌 상태가 되는 듯 (현재 동작중인 상황)

In [8]:
import time

def wait_on_run(run, thread):
    while run.status == "queued" or run.status == "in_progress":
        run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id,
        )
        time.sleep(0.5)
    return run

<font color=green> run중이면 return하지 않고 있다가, 완료가 되면 return 하게 됨</font>
- 완료가 되었기 때문에 return됨

In [9]:
run = wait_on_run(run, thread)
show_json(run)

C:\Users\jjsd4\AppData\Local\Temp\ipykernel_32260\3504579710.py:5: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run = client.beta.threads.runs.retrieve(


{'id': 'run_VM0izCrE2IAuBcYNdEtUSkWG',
 'assistant_id': 'asst_ABxisLXuhXXBuVLYrk3sJJlR',
 'cancelled_at': None,
 'completed_at': 1753185067,
 'created_at': 1753185064,
 'expires_at': None,
 'failed_at': None,
 'incomplete_details': None,
 'instructions': '너는 개인 수학 교사야. 질문에 한 문장 이하로 짧게 답해줘',
 'last_error': None,
 'max_completion_tokens': None,
 'max_prompt_tokens': None,
 'metadata': {},
 'model': 'gpt-4o',
 'object': 'thread.run',
 'parallel_tool_calls': True,
 'required_action': None,
 'response_format': 'auto',
 'started_at': 1753185065,
 'status': 'completed',
 'thread_id': 'thread_D97HMuorDUdCSxehSzzucm02',
 'tool_choice': 'auto',
 'tools': [],
 'truncation_strategy': {'type': 'auto', 'last_messages': None},
 'usage': {'completion_tokens': 6,
  'prompt_tokens': 68,
  'total_tokens': 74,
  'prompt_token_details': {'cached_tokens': 0},
  'completion_tokens_details': {'reasoning_tokens': 0}},
 'temperature': 1.0,
 'top_p': 1.0,
 'tool_resources': {},
 'reasoning_effort': None}

## <font color=yellow>4. Messages</font>
- Run이 완료되었으므로, Assistant가 무엇을 추가했는지 보기 위해 Thread 안의 메시지들을 나열할 수 있음

In [10]:
messages = client.beta.threads.messages.list(thread_id=thread.id)
show_json(messages)

C:\Users\jjsd4\AppData\Local\Temp\ipykernel_32260\3890215631.py:1: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  messages = client.beta.threads.messages.list(thread_id=thread.id)


{'data': [{'id': 'msg_qZB88CYpEVrFZY790qesjwiu',
   'assistant_id': 'asst_ABxisLXuhXXBuVLYrk3sJJlR',
   'attachments': [],
   'completed_at': None,
   'content': [{'text': {'annotations': [], 'value': 'x = 1'},
     'type': 'text'}],
   'created_at': 1753185066,
   'incomplete_at': None,
   'incomplete_details': None,
   'metadata': {},
   'object': 'thread.message',
   'role': 'assistant',
   'run_id': 'run_VM0izCrE2IAuBcYNdEtUSkWG',
   'status': None,
   'thread_id': 'thread_D97HMuorDUdCSxehSzzucm02'},
  {'id': 'msg_KMG3hwEwYfIbQJTyp4MFuY7j',
   'assistant_id': None,
   'attachments': [],
   'completed_at': None,
   'content': [{'text': {'annotations': [],
      'value': "방정식 '3x + 11 = 14'를 풀어줘"},
     'type': 'text'}],
   'created_at': 1753185047,
   'incomplete_at': None,
   'incomplete_details': None,
   'metadata': {},
   'object': 'thread.message',
   'role': 'user',
   'run_id': None,
   'status': None,
   'thread_id': 'thread_D97HMuorDUdCSxehSzzucm02'}],
 'has_more': False,
 

보시다시피 메시지들은 역시간 순서로 정렬되어 있음 
- 첫번째 딕셔너리가 최근 답변한 것임
- 이렇게 함으로써 가장 최근의 결과가 항상 첫 번째 '페이지'에 있게 됨 (결과는 페이지네이션 될 수 있으므로) 
- 이는 Chat Completions API의 메시지 순서와 반대이므로 주의해야 함

### <font color=green>모델에서 풀이과정을 더 설명해달라고 요청해보겠음</font>

In [11]:
# Create a message to append to our thread
message = client.beta.threads.messages.create(
    thread_id=thread.id, role="user", content="설명해 주시겠어요?"
)

# Execute our run
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
)

# Wait for completion
wait_on_run(run, thread)

# Retrieve all the messages added after our last user message
messages = client.beta.threads.messages.list(
    thread_id=thread.id, order="asc", after=message.id
)
show_json(messages)

C:\Users\jjsd4\AppData\Local\Temp\ipykernel_32260\857599754.py:2: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  message = client.beta.threads.messages.create(
C:\Users\jjsd4\AppData\Local\Temp\ipykernel_32260\857599754.py:7: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run = client.beta.threads.runs.create(
C:\Users\jjsd4\AppData\Local\Temp\ipykernel_32260\3504579710.py:5: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run = client.beta.threads.runs.retrieve(
C:\Users\jjsd4\AppData\Local\Temp\ipykernel_32260\857599754.py:16: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  messages = client.beta.threads.messages.list(


{'data': [{'id': 'msg_toqXm1vuWdmfl4FAsXy0YPmo',
   'assistant_id': 'asst_ABxisLXuhXXBuVLYrk3sJJlR',
   'attachments': [],
   'completed_at': None,
   'content': [{'text': {'annotations': [],
      'value': '양쪽에서 11을 빼서 3x = 3을 만든 후, 3으로 나누면 x = 1입니다.'},
     'type': 'text'}],
   'created_at': 1753185321,
   'incomplete_at': None,
   'incomplete_details': None,
   'metadata': {},
   'object': 'thread.message',
   'role': 'assistant',
   'run_id': 'run_S5ZyPR7DHPAnLCBy7I4dvGTv',
   'status': None,
   'thread_id': 'thread_D97HMuorDUdCSxehSzzucm02'}],
 'has_more': False,
 'object': 'list',
 'first_id': 'msg_toqXm1vuWdmfl4FAsXy0YPmo',
 'last_id': 'msg_toqXm1vuWdmfl4FAsXy0YPmo'}

<font color=red>이 코드를 거의 변경하지 않고도, 우리의 Assistant에 매우 강력한 기능을 추가할 수 있는 방법을 곧 보게 될것임</font>